In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import datetime
import os.path
import backtrader as bt
import numpy as np
from pprint import pformat
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import sys

# Strategy

In [ ]:

# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod_sma', 15),
        ('maperiod_rsi', 21),
        ('rsi_low', 50),
        ('rsi_high', 60),
        ('printlog', False),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open
        #self.dataNOopen = self.data.open ##Both are same
        #self.dataNOclose = self.data.close ##Both are same

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.maperiod_sma)
        
        # Add a RSI indicator
        self.rsi = bt.indicators.RSI_SMA(self.datas[0].close, period=self.params.maperiod_rsi)
        
            
        # Indicators for the plotting show
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25,
                                            subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        #rsi = bt.indicators.RSI(self.datas[0])
        #bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)
        

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected', doprint=True)

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f,      NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Open : %.2f, Close : %.2f , SMA : %.2f, RSI : %.2f' % (self.dataopen[0],self.dataclose[0],self.sma[0],self.rsi[0]))
  
        
        #print('Self dataNOopen',self.dataNOopen[0]) Both are same
        #print('DatasdataNOclose0',self.dataNOclose[0]) Both are same

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if (self.dataclose[0] > self.sma[0]) and (self.rsi < self.params.rsi_low) :
            #if (self.dataclose[0] > self.sma[0])  :

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            #if (self.dataclose[0] < self.sma[0]) and (self.rsi > self.params.rsi_high) :
            if (self.dataclose[0] < self.sma[0]) :
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('(MA Period %2d)  (RSI Period %2d) (RSI Low %2d) (RSI High %2d) Ending Value %.2f' %
                 (self.params.maperiod_sma,self.params.maperiod_rsi,self.params.rsi_low,self.params.rsi_high,self.broker.getvalue()), doprint=True)



# Analyzer Functions

In [ ]:
def printTradeAnalysis(analyzer):
    '''
    Function to print the Technical Analysis results in a nice format.
    '''
    #Get the results we are interested in
    total_open = analyzer.total.open
    total_closed = analyzer.total.closed
    total_won = analyzer.won.total
    total_lost = analyzer.lost.total
    win_streak = analyzer.streak.won.longest
    lose_streak = analyzer.streak.lost.longest
    pnl_net = round(analyzer.pnl.net.total,2)
    strike_rate = round((total_won / total_closed) * 100,2)
    #Designate the rows
    h1 = ['Total Open', 'Total Closed', 'Total Won', 'Total Lost']
    h2 = ['Strike Rate','Win Streak', 'Losing Streak', 'PnL Net']
    r1 = [total_open, total_closed,total_won,total_lost]
    r2 = [strike_rate, win_streak, lose_streak, pnl_net]
    #Check which set of headers is the longest.
    if len(h1) > len(h2):
        header_length = len(h1)
    else:
        header_length = len(h2)
    #Print the rows
    print_list = [h1,r1,h2,r2]
    row_format ="{:<15}" * (header_length + 1)
    print("Trade Analysis Results:")
    for row in print_list:
        print(row_format.format('',*row))
        
def printSQN(analyzer):
    sqn = round(analyzer.sqn,2)
    print('SQN: {}'.format(sqn))

def printSharpeRatio(analyzer):
    SRatio = analyzer
    print('Sharpe Ratio: {}'.format(SRatio))
    
def printDrawDown(analyzer):
    DDown=round(analyzer,4)
    print('Draw Down: {}'.format(DDown))
    
    

In [ ]:
class DecisiveAnalyzer(bt.Analyzer):
    
    def __init__(self):
        self.maperiod_sma = self.strategy.params.maperiod_sma
        self.maperiod_rsi = self.strategy.params.maperiod_rsi
        self.rsi_low = self.strategy.params.rsi_low
        self.rsi_high = self.strategy.params.rsi_high
        self.equity = []
        self.equitydf = pd.DataFrame()


    def start(self):
        # Not needed ... but could be used
        self.init_cash = self.strategy.broker.cash 
        self.num_trades = 0
        self.first_trade_open = None
        
    def next(self):
        # Not needed ... but could be used
        pass

    def notify_trade(self, trade):
        if not self.first_trade_open:
            self.first_trade_open = self.strategy.datetime.datetime()
            self.equitydf.at[self.first_trade_open, 'equity'] = self.init_cash

        if trade.isclosed:
            self.num_trades += 1
            self.equitydf.at[self.strategy.datetime.datetime(), 'equity'] = self.strategy.broker.getvalue()
        
        
    def stop(self):
        self.final_cash = self.strategy.broker.cash
        self.final_val = self.strategy.broker.get_value()

        
    def get_analysis(self):
        # Add trade profit to dataframe
        self.equitydf['net_profit'] = self.equitydf['equity'].diff()

        outp = {
            'params_sma': (self.maperiod_sma),
            'params_rsi': (self.maperiod_rsi),
            'params_rsi_low': (self.rsi_low),
            'params_rsi_high': (self.rsi_high),
            'profit': self.final_val - self.init_cash,
            'num_trades': self.num_trades,
            'equitydf': self.equitydf,
        }

        return outp


In [ ]:
def best_result_from_cerebro_opti_run(result):
    params_sma  = []
    params_rsi  = []
    params_rsi_low  = []
    params_rsi_high  = []
    n_trades = []
    profit   = []
    trades   = []
    for res in result:
        r = res[0].analyzers.decisive.get_analysis()
        params_sma.append(r['params_sma'])
        params_rsi.append(r['params_rsi'])
        params_rsi_low.append(r['params_rsi_low'])
        params_rsi_high.append(r['params_rsi_high'])
        n_trades.append(r['ntrade'])
        profit.append(r['profit'])
        trades.append(r['trades'])

    prof_ind = np.argmax(profit) 
    best_params_sma = params_sma[prof_ind]
    best_params_rsi = params_rsi[prof_ind]
    best_params_rsi_low = params_rsi_low[prof_ind]
    best_params_rsi_high = params_rsi_high[prof_ind]
    best_profit = profit[prof_ind]
    best_ntrades = n_trades[prof_ind]
    best_trades = trades[prof_ind]
    
    print('best:{} profit:{} trades:{}'.format(best_params_sma,best_params_rsi,best_params_rsi_high,best_params_rsi_low, best_profit, best_ntrades))
    return (best_params_sma,best_params_rsi,best_params_rsi_low,best_params_high, best_profit, best_ntrades, best_trades)

In [ ]:
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

#df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = pd.read_parquet('D:/study/dalpha/DecisiveWorkflowResearch-master/data/processed/5min/CL_5min_20160103_20190405.parquet')
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()
df['2017-01-01':'2017-04-01']['close'].plot()

In [ ]:
periods = [
{'run': 0, 'oos': ('2017-04-01', '2017-05-01'), 'is': ('2017-01-01', '2017-04-01')},
{'run': 1, 'oos': ('2017-05-01', '2017-06-01'), 'is': ('2017-02-01', '2017-05-01')},
{'run': 2, 'oos': ('2017-06-01', '2017-07-01'), 'is': ('2017-03-01', '2017-06-01')},
{'run': 3, 'oos': ('2017-07-01', '2017-08-01'), 'is': ('2017-04-01', '2017-07-01')},
{'run': 4, 'oos': ('2017-08-01', '2017-09-01'), 'is': ('2017-05-01', '2017-08-01')},
{'run': 5, 'oos': ('2017-09-01', '2017-10-01'), 'is': ('2017-06-01', '2017-09-01')},
{'run': 6, 'oos': ('2017-10-01', '2017-11-01'), 'is': ('2017-07-01', '2017-10-01')},
{'run': 7, 'oos': ('2017-11-01', '2017-12-01'), 'is': ('2017-08-01', '2017-11-01')},
{'run': 8, 'oos': ('2017-12-01', '2018-01-01'), 'is': ('2017-09-01', '2017-12-01')},
{'run': 9, 'oos': ('2018-01-01', '2018-02-01'), 'is': ('2017-10-01', '2018-01-01')},
{'run': 10, 'oos': ('2018-02-01', '2018-03-01'), 'is': ('2017-11-01', '2018-02-01')},
{'run': 11, 'oos': ('2018-03-01', '2018-04-01'), 'is': ('2017-12-01', '2018-03-01')},
{'run': 12, 'oos': ('2018-04-01', '2018-05-01'), 'is': ('2018-01-01', '2018-04-01')},
{'run': 13, 'oos': ('2018-05-01', '2018-06-01'), 'is': ('2018-02-01', '2018-05-01')},
]

# TestStrategy - In Sample Execution  

In [ ]:
start_date = periods[0]['is'][0]
end_date = periods[-1]['is'][1]
print('Start/End: {} - {}'.format(start_date, end_date))

cerebro = bt.Cerebro()

# Add a strategy
strats = cerebro.optstrategy(
    TestStrategy,
    maperiod_sma=(15,21,26),
    maperiod_rsi=(14,21),
    rsi_low=(50),
    rsi_high=(60)
    )

cerebro.addanalyzer(DecisiveAnalyzer, _name='decisive')

cerebro.optreturn = False
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
cerebro.broker.setcommission(commission=0.0)

data = bt.feeds.PandasData(dataname = df[start_date:end_date])
cerebro.adddata(data)

results = cerebro.run(maxcpus=1) 

In [ ]:
# Get in-sample and out-of-sample best parameters with correct pre-allocation
for period in periods:
    # Get the insample time period
    insample_start = period['is'][0]
    insample_end = period['is'][1]
    print('In-Sample Period: {} - {}'.format(insample_start, insample_end))
    
    # Put the best parameters and best profit into a list
    period_netprofit = []
    for result in results:
        r = result[0].analyzers.decisive.get_analysis()
        df = r['equitydf']
        maperiod_sma = r['params_sma']
        maperiod_rsi = r['params_rsi']
        rsi_low = r['params_rsi_low']
        rsi_high = r['params_rsi_high']
        period_profit = df[insample_start:insample_end]['net_profit'].sum()
        #print('\tmaperiod_sma : {} \tmaperiod_rsi : {} \t Profit : {}'.format(maperiod_sma,maperiod_rsi, period_profit))
        period_netprofit.append((maperiod_sma,maperiod_rsi, period_profit))
        
    # Get the best profit parameters
    max_set = max(period_netprofit, key=lambda x:x[2])
    maperiod_sma,maperiod_rsi, max_profit = max_set
    
    # Add it to the dictionary
    print('\t\tBest is:- SMA : {} RSI : {} with {} profit'.format(maperiod_sma,maperiod_rsi, max_profit))
    period['best_insample_maperiod_sma'] = maperiod_sma
    period['best_insample_maperiod_rsi'] = maperiod_rsi
    period['best_insample_profit'] = max_profit

In [ ]:
def days_from_date_tuple(mytuple):
    """Get days between dates to annualize"""
    days_start = datetime.datetime.strptime(mytuple[0], '%Y-%m-%d')
    days_end = datetime.datetime.strptime(mytuple[1], '%Y-%m-%d')
    days = (days_end - days_start).days
    return days

# Get in-sample and out-of-sample best parameters with correct pre-allocation
all_walkforward_efficiency = []
for period in periods:
    # Get the insample time period
    oos_start = period['oos'][0]
    oos_end = period['oos'][1]
    insample_profit = period['best_insample_profit']
    print('Out-of-Sample Period: {} - {}'.format(oos_start, oos_end))
    
    # Find the set of results that has the same best parameters
    target_result = None
    for result in results:
        r = result[0].analyzers.decisive.get_analysis()
        if (r['params_sma'] == period['best_insample_maperiod_sma']) and (r['params_rsi'] == period['best_insample_maperiod_rsi']) :
            target_result = r
            break
    
    # Get the equity curve of this best parameter
    target_df = target_result['equitydf']
    
    # Splice to the OOS date range only
    oos_profit = target_df[oos_start:oos_end]['net_profit'].sum()
    
    # Annualize the profit
    insample_days = days_from_date_tuple(period['is'])
    oos_days = days_from_date_tuple(period['oos'])

    insample_annual_profit = 365/insample_days * insample_profit
    oos_annual_profit = 365/oos_days * oos_profit
 
    # Calculate WFE
    period_wfe = insample_annual_profit * 100 / oos_annual_profit
    period['wfe'] = period_wfe
    all_walkforward_efficiency.append(period_wfe)
    
    print('\t Period Profit IS:{:0.2f} / OOS:{:0.2f}'.format(insample_profit, oos_profit))
    print('\t Annual Profit IS:{:0.2f} / OOS:{:0.2f}'.format(insample_annual_profit, oos_annual_profit))

    print('\t WFE: {:0.1f}%'.format(period_wfe))
    print('\t Paramters SMA : {} RSI :{}'.format(target_result['params_sma'],target_result['params_rsi']))

wfe = sum(all_walkforward_efficiency) / len(all_walkforward_efficiency) 
print('Overall WFE: {:0.1f}%'.format(wfe))

In [ ]:
# Create a Stratey
class WalkforwardStrategy(bt.Strategy):
    params = (
        ('maperiod_sma', 26),
        ('maperiod_rsi', 21),
        ('rsi_low', 50),
        ('rsi_high', 60),
        ('printlog', False),
        ('live', False),
        ('walkforward', None),        
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.dataopen = self.datas[0].open
        #self.dataNOopen = self.data.open ##Both are same
        #self.dataNOclose = self.data.close ##Both are same

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.maperiod_sma)
        
        # Add a RSI indicator
        self.rsi = bt.indicators.RSI_SMA(self.datas[0].close, period=self.params.maperiod_rsi)
        
        #Trim the indicators if we are running live, just need the current one
        if not self.params.live:
            pass
       
        self.current_row = None
        self.wfsma = []

        if self.params.walkforward:
              for run in self.params.walkforward:
                  self.wfsma.append({
                    'sma': bt.indicators.SimpleMovingAverage(self.datas[0], period=run['best_insample_maperiod_sma']),
                    'rsi': bt.indicators.RSI_SMA(self.datas[0], period=run['best_insample_maperiod_rsi']),
                    'test_period': run['oos'],
                    'train_param_sma': run['best_insample_maperiod_sma'],
                    'train_param_rsi': run['best_insample_maperiod_rsi'],
                }) 

        print(pformat(self.wfsma))
        
        
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected', doprint=True)

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f,      NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Open : %.2f, Close : %.2f , SMA : %.2f, RSI : %.2f' % (self.dataopen[0],self.dataclose[0],self.sma[0],self.rsi[0]))
  
        # Walk-forward logic
        for row in self.wfsma:
            start, end = row['test_period']
            period_start = datetime.datetime.strptime(start, '%Y-%m-%d')
            period_end = datetime.datetime.strptime(end, '%Y-%m-%d')
            if self.datetime.datetime() >= period_start and self.datetime.datetime() < period_end:
                self.current_row = row
        
        #print('Self dataNOopen',self.dataNOopen[0]) Both are same
        #print('DatasdataNOclose0',self.dataNOclose[0]) Both are same

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if (self.dataclose[0] > self.sma[0]) and (self.rsi < self.params.rsi_low) :
            #if (self.dataclose[0] > self.sma[0])  :

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            #if (self.dataclose[0] < self.sma[0]) and (self.rsi > self.params.rsi_high) :
            if (self.dataclose[0] < self.sma[0]) :
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('(MA Period %2d)  (RSI Period %2d) (RSI Low %2d) (RSI High %2d) Ending Value %.2f' %
                 (self.params.maperiod_sma,self.params.maperiod_rsi,self.params.rsi_low,self.params.rsi_high,self.broker.getvalue()), doprint=True)



In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(WalkforwardStrategy, 
                    maperiod_sma=15,
                    maperiod_rsi=21,
                    walkforward=periods,
                    live=False)

# Load data
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

#df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = pd.read_parquet('D:/study/dalpha/DecisiveWorkflowResearch-master/data/processed/5min/CL_5min_20160103_20190405.parquet')
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()

# periods
start_date = periods[0]['oos'][0]
end_date = periods[-1]['oos'][1]

print('Start: {} End: {}'.format(start_date, end_date))
data = bt.feeds.PandasData(dataname = df[start_date:end_date])


# Add the Data Feed to Cerebro
cerebro.adddata(data)
cerebro.addanalyzer(DecisiveAnalyzer, _name='decisive')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', timeframe=bt.TimeFrame.Months)

# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
results = cerebro.run()



In [ ]:
printTradeAnalysis(results[0].analyzers.ta.get_analysis())
printSQN(results[0].analyzers.sqn.get_analysis())
print(results[0].analyzers.sharpe.get_analysis()['sharperatio'])



In [ ]:
# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
r = results[0].analyzers.decisive.get_analysis()
print(r)
cumsum = r['equitydf']['net_profit'].cumsum()
print(cumsum)
plt.figure(figsize=(10,5))
plt.plot(cumsum)
plt.title('Walkforward Equity Curve')
plt.xlabel('Trades')
plt.ylabel('Equity')

# cerebro.plot(volume=False, iplot=False)

In [ ]:
avg_wfe = np.round(np.asarray(all_walkforward_efficiency).mean(),1)
print('Average WFE: {}%'.format(avg_wfe))

In [ ]:
print(sys.path)
sys.path.append('D:/study/dalpha/DecisiveWorkflowResearch-master/decisivealpha') #won't work outside my machine.
from decisivealpha.montecarlo import MonteCarlo

In [ ]:
# Assume CL futures margin
margin = 5000

start_date = datetime.date(2016, 1, 1)
end_date = datetime.date(2018, 1, 1)

In [ ]:
# We're going to generate a random list of trades,
# and you should replace this with your walkforward list of trades
print(r['equitydf']['net_profit'].values[1:])
mc = MonteCarlo(r['equitydf']['net_profit'].values[1:])

# We will sample with replacement the number of trades per year
# so we need the start and end date to determine how many trades at in a year on average
mc.settings(margin, start_date, end_date)

# Test different levels of equity starting at this value
trial_starting_equity = int(margin * 1.5)

# Run the Monte Carlo
results = mc.run(trial_starting_equity)

In [ ]:
# Put the results in a dataframe so it's nicer to look at in notebook
# Our goal is to get the highest equity below 10% Risk of Ruin
df = pd.DataFrame(index=range(1,len(results)))
count = 1
for result in results:
    df.loc[count, 'equity'] = result['equity']
    df.loc[count, 'is_ruined'] = result['is_ruined']
    df.loc[count, 'is_profitable'] = result['is_profitable']
    df.loc[count, 'returns_pct'] = result['returns_pct']
    df.loc[count, 'drawdown_pct'] = result['drawdown_pct']
    df.loc[count, 'returns_per_drawdown'] = result['returns_per_drawdown']
    count += 1

# Get the recommended values
recommended = df[df['is_ruined'] <= 10].iloc[0]
print('Recommend a starting equity of {}, which has {:0.2}% Risk-of-Ruin, \n\t{:0.0f}% Probability-of-Profit and a {:0.2f} Returns/Drawdown Ratio'.format(
                recommended['equity'], recommended['is_ruined'], 
                recommended['is_profitable'], recommended['returns_per_drawdown']))

if recommended['is_ruined'] > 10 or recommended['returns_per_drawdown'] < 2.0:
    print("Risk Assessment: FAILED")
else:
    print("Risk Assessment: PASSED")

mc_1p5x = recommended['drawdown_pct'] * 1.5
print("MC-Drawdown: {:0.1f}% MC-1.5x-DD: {:0.1f}%".format(recommended['drawdown_pct'], mc_1p5x))

profit = recommended['equity'] * recommended['returns_pct'] / 100
months = (end_date - start_date).days/30
average_monthly_net_profit = profit / months
print("Average monthly net profit: {:0.1f}".format(average_monthly_net_profit))

df